# Phase 2: Filling Missing Values

## Columns:

**Categorical Columns**  
- [] make_model
- [] body_type
- [] vat
- [] registration_year
- [] previous_owners
- [] type
- [] next_inspection
- [] inspection_new
- [] body_color
- [] paint_type
- [] upholstery_color
- [] upholstery_material
- [] gearing_type
- [] fuel_type
- [] particulate
- [] co2_emission
- [] emission_class
- [] drive_chain
- [] consumption_country
- [] consumption_city
- [] consumption_combined
- [] entertainment_media
- [] safety_security
- [] comfort_convenience
- [] extras
 
**Quantitative Columns**
- [] price
- [] km
- [] hp
- [] displacement
- [] warranty
- [] weight
- [] nr_of_doors
- [] nr_of_seats
- [] cylinders
- [] gears


---

In [1144]:
import pandas as pd
import numpy as np

df = pd.read_csv("cleaned_car_data.csv")

In [1145]:
df.head(3).T

,0,1,2
make_model,Audi A1,Audi A1,Audi A1
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,56013.0,80000.0,83450.0
hp,66.0,141.0,85.0
type,Used,Used,Used
previous_owners,2.0,NaN,1.0
next_inspection,2021.0,NaN,NaN
inspection_new,Yes,NaN,NaN


In [1146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   make_model            15919 non-null  object 
 1   body_type             15859 non-null  object 
 2   price                 15919 non-null  int64  
 3   vat                   11406 non-null  object 
 4   km                    14895 non-null  float64
 5   hp                    15831 non-null  float64
 6   type                  15917 non-null  object 
 7   previous_owners       9254 non-null   float64
 8   next_inspection       2825 non-null   float64
 9   inspection_new        3570 non-null   object 
 10  warranty              4853 non-null   float64
 11  full_service          8215 non-null   object 
 12  non-smoking_vehicle   7177 non-null   object 
 13  null                  15919 non-null  object 
 14  offer_number          12744 non-null  object 
 15  first_registration 

---

## Fill Missing Values Function


### use q cut on numerical to turn into categorical.
### also use other methods

In [1147]:
def fill(method, df, column, group_cols=None):
   """
   Fills NaN values in `df[column]` either using the overall mean, median or mode (no grouping)
   or group-specific mean, median or mode (group_col provided).
   Prints stats about how many NaNs were filled and the final distribution.
   """
   # Make a copy of the dataframe to avoid modification warnings
   working_df = df.copy()
   
   # Debug prints: which column is being filled, and grouping info.
   print('Filling column:', column)
   print('Grouping by:', group_cols)
   
   # 1. Count NaNs before filling
   nan_before = working_df[column].isnull().sum()
   
   # 2. Fill logic
   if method in ['mean', 'median']:
       if group_cols is None:
           # Fill all NaN in 'column' with the overall mean/median
           working_df[column].fillna(getattr(working_df[column], method)(), inplace=True)
       else:
           # Calculate groupwise mean/median for each row
           group = working_df.groupby(group_cols)[column].transform(method)
           # Fill missing values in df[column] with corresponding group mean/median
           working_df[column].fillna(group, inplace=True)
   
   elif method == 'mode':
       if group_cols is None:
           # Fill all NaN in 'column' with the overall mode
           working_df[column].fillna(working_df[column].mode().iloc[0], inplace=True)            
       else:
           # Calculate groupwise mode for each row
           group = working_df.groupby(group_cols)[column]\
           .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)
           # Fill missing values in df[column] with corresponding group mode
           working_df[column].fillna(group, inplace=True)
   
   # 3. Count NaNs after filling
   nan_after = working_df[column].isnull().sum()
   nan_filled = nan_before - nan_after
   
   # 4. Print final stats
   print("Number of NaN before filling:", nan_before)
   print("Number of NaN filled:", nan_filled)
   print("Number of NaN after filling:", nan_after)
   print("------------------")
   print(working_df[column].value_counts(dropna=False))
   
   return working_df

---

## Provided Functions

In [1148]:
def fill_most_freq(df, group_col, col_name):
    
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != [] :
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [1149]:
def fill_prop(df, group_col, col_name):
    
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [1150]:
def double_stage(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != [] :
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != [] :
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

### make_model

---

## Categorical Columns
- [x] make_model
- [x] body_type
- [x] vat
- [x] registration_year
- [x] previous_owners
- [] type
- [x] next_inspection
- [x] inspection_new
- [x] body_color
- [x] paint_type
- [x] upholstery_color
- [] upholstery_material
- [x] gearing_type
- [] fuel_type
- [] particulate
- [x] co2_emission
- [x] emission_class
- [x] drive_chain
- [x] entertainment_media
- [x] safety_security
- [x] comfort_convenience
- [x] extras

In [1151]:
df['make_model'].value_counts(dropna=False)

make_model
Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: count, dtype: int64

### body_type

In [1152]:
df['body_type'].value_counts(dropna=False)

body_type
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

In [1153]:
df['make_model'].value_counts(dropna=False)

make_model
Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: count, dtype: int64

#### same make_model same body_type so we can use fill()

In [1154]:
df = fill('mode', df, 'body_type', 'make_model')

Filling column: body_type
Grouping by: make_model
Number of NaN before filling: 60
Number of NaN filled: 60
Number of NaN after filling: 0
------------------
body_type
Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64


### vat

In [1155]:
df.vat.value_counts(dropna=False)

vat
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: count, dtype: int64

In [1156]:
df['vat'] = df['vat'].fillna('VAT undeductible') 

* All null values of vat column assigned as 'VAT undeductible'

In [1157]:
df.vat.value_counts(dropna=False)

vat
VAT deductible      10980
VAT undeductible     4513
Price negotiable      426
Name: count, dtype: int64

### registration_year (converted to age)

In [1158]:
df.loc[((df['type'] == 'New') & df.registration_year.isnull()), 'registration_year']= 2019

 - #### new cars become 2019

In [1159]:
df.groupby('registration_year').km.mean()

registration_year
2016.0    77442.520958
2017.0    41754.940709
2018.0    18035.239072
2019.0     1653.107634
Name: km, dtype: float64

In [1160]:
df.loc[(df['registration_year'].isnull()) & (df.km <= 5000),'registration_year'] =2019

In [1161]:
df.loc[(df['registration_year'].isnull()) & (df.km > 50000),'registration_year'] =2016

In [1162]:
df.loc[(df['registration_year'].isnull()) & (df.km < 19000),'registration_year'] =2018

In [1163]:
df.loc[(df['registration_year'].isnull()) & (df.km < 40000),'registration_year'] =2017

- #### otherwise filled based on mileage

In [1164]:
df[(df['make_model'] == 'Audi A3') & (df['body_type'] == 'Sedans')].groupby('registration_year').price.mean()

registration_year
2016.0    16702.052388
2017.0    19970.030631
2018.0    22544.715092
2019.0    24859.973913
Name: price, dtype: float64

In [1165]:
df[(df['make_model'] == 'Opel Insignia')].groupby('registration_year').price.mean()

registration_year
2016.0    13606.709507
2017.0    16995.140917
2018.0    21390.167750
2019.0    32103.197015
Name: price, dtype: float64

In [1166]:
df.loc[(df['registration_year'].isnull())]

,make_model,body_type,price,vat,km,hp,type,previous_owners,next_inspection,inspection_new,...,extras,safety_security,gears,registration_year,upholstery_material,upholstery_color,particulate,combined_consumption,city_consumption,country_consumption
5237,Audi A3,Sedans,25400,VAT undeductible,NaN,85.0,NaN,NaN,NaN,NaN,...,['Alloy wheels'],"['ABS', 'Central door lock', 'Driver-side airb...",7.0,NaN,Cloth,NaN,unparticulate,3.9,4.1,3.7
5329,Audi A3,Sedans,24900,VAT undeductible,NaN,85.0,Pre-registered,NaN,NaN,NaN,...,['Alloy wheels'],"['ABS', 'Central door lock', 'Driver-side airb...",7.0,NaN,Cloth,NaN,unparticulate,3.9,4.1,3.7
12550,Opel Insignia,Sedans,33800,VAT undeductible,NaN,100.0,Employee's car,NaN,NaN,NaN,...,"['Alloy wheels', 'Sport package', 'Touch scree...","['ABS', 'Blind spot monitor', 'Central door lo...",6.0,NaN,Part leather,Black,unparticulate,5.4,7.0,4.6
12882,Opel Insignia,Station wagon,31318,VAT deductible,NaN,100.0,Used,NaN,NaN,NaN,...,"['Alloy wheels', 'Sport seats']","['ABS', 'Central door lock', 'Driver-side airb...",6.0,NaN,Full leather,Black,unparticulate,4.3,NaN,NaN


In [1167]:
df['registration_year']=df['registration_year'].fillna(2019)

In [1168]:
df['registration_year'] = 2019 - df['registration_year']

df = df.rename(columns = {'registration_year': 'age'})

In [1169]:
df['age'].value_counts(dropna=False)

age
1.0    4525
0.0    4434
3.0    3679
2.0    3281
Name: count, dtype: int64

- #### year column converted to age column

### previous owners

In [1170]:
index=df[df["km"]<10].index

In [1171]:
df.loc[index,"previous_owners"]=0.0

In [1172]:
df.loc[index,"previous_owners"]=0.0

In [1173]:
df.loc[(df["type"]=="New") & (df["previous_owners"].isnull()),"previous_owners"]=1.0

In [1174]:
df.loc[((df["age"]==0) | (df["km"]<5000)) & (df["previous_owners"].isnull()),"previous_owners"]=1.0

In [1175]:
mode_group_po = df.groupby(['age'])['previous_owners'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [1176]:
df["previous_owners"].fillna(mode_group_po, inplace=True)

In [1177]:
df["previous_owners"].value_counts(dropna=False)

previous_owners
1.0    14287
0.0      835
2.0      778
3.0       17
4.0        2
Name: count, dtype: int64

### type

In [1178]:
df['type'].value_counts(dropna=False)

type
Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: count, dtype: int64

#### Only 2 NaN so replaced with mode

In [1179]:
df = fill('mode', df, 'type')

Filling column: type
Grouping by: None
Number of NaN before filling: 2
Number of NaN filled: 2
Number of NaN after filling: 0
------------------
type
Used              11098
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: count, dtype: int64


### next_inspection (dropped)

In [1180]:
df['next_inspection'].value_counts(dropna=False)

next_inspection
NaN       13094
2021.0     1401
2020.0      557
2022.0      483
2019.0      336
2018.0       26
2017.0        7
2023.0        5
2001.0        5
2016.0        3
2014.0        1
1921.0        1
Name: count, dtype: int64

In [1181]:
df.drop(["next_inspection"],axis=1,inplace=True)

### inspection_new

In [1182]:
df['inspection_new'].fillna('No', inplace=True)

In [1183]:
df['inspection_new'].value_counts(dropna=False)

inspection_new
No     12349
Yes     3570
Name: count, dtype: int64

### body_color

In [1184]:
df['body_color'].value_counts(dropna=False)

body_color
Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: count, dtype: int64

In [1185]:
df = fill('mode', df, 'body_color', ['make_model', 'body_type'])

Filling column: body_color
Grouping by: ['make_model', 'body_type']
Number of NaN before filling: 597
Number of NaN filled: 594
Number of NaN after filling: 3
------------------
body_color
Black     3941
Grey      3836
White     3469
Silver    1647
Blue      1431
Red        957
Brown      289
Green      158
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
NaN          3
Gold         2
Name: count, dtype: int64


In [1187]:
df = fill('mode', df, 'body_color', ['make_model'])

Filling column: body_color
Grouping by: ['make_model']
Number of NaN before filling: 3
Number of NaN filled: 3
Number of NaN after filling: 0
------------------
body_color
Black     3941
Grey      3839
White     3469
Silver    1647
Blue      1431
Red        957
Brown      289
Green      158
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: count, dtype: int64


### paint_type

In [1188]:
df = fill('mode', df, 'paint_type', ['make_model', 'body_type'])

Filling column: paint_type
Grouping by: ['make_model', 'body_type']
Number of NaN before filling: 5772
Number of NaN filled: 5761
Number of NaN after filling: 11
------------------
paint_type
Metallic       15512
Uni/basic        390
NaN               11
Perl effect        6
Name: count, dtype: int64


In [1189]:
df = fill('mode', df, 'paint_type', 'make_model')

Filling column: paint_type
Grouping by: make_model
Number of NaN before filling: 11
Number of NaN filled: 11
Number of NaN after filling: 0
------------------
paint_type
Metallic       15523
Uni/basic        390
Perl effect        6
Name: count, dtype: int64


### upholstery_color

In [1092]:
df.drop(['upholstery_color'], axis=1, inplace=True)

### upholstery_material

In [1093]:
df['upholstery_material'].value_counts(dropna=False)

upholstery_material
Cloth           8423
NaN             4503
Part leather    1499
Full leather    1009
Other            368
Velour            60
alcantara         57
Name: count, dtype: int64

### gearing_type

In [1094]:
df['gearing_type'].value_counts(dropna=False)

gearing_type
Manual            8153
Automatic         7297
Semi-automatic     469
Name: count, dtype: int64

### fuel

In [1095]:
df['fuel']

0          d
1          d
2          d
3          d
4          d
        ... 
15914    NaN
15915      d
15916      -
15917      -
15918      o
Name: fuel, Length: 15919, dtype: object

### co2_emission

In [1096]:
df['co2_emission'].value_counts(dropna=False)

co2_emission
NaN      2436
120.0     740
99.0      545
97.0      537
104.0     501
         ... 
51.0        1
165.0       1
331.0       1
80.0        1
193.0       1
Name: count, Length: 120, dtype: int64

In [1190]:
df = fill("mode", df, 'co2_emission', ['make_model', 'displacement'])

Filling column: co2_emission
Grouping by: ['make_model', 'displacement']
Number of NaN before filling: 2436
Number of NaN filled: 1950
Number of NaN after filling: 486
------------------
co2_emission
120.0    984
97.0     773
99.0     692
114.0    577
119.0    556
        ... 
51.0       1
165.0      1
160.0      1
80.0       1
193.0      1
Name: count, Length: 120, dtype: int64


In [1191]:
df = fill("mode", df, 'co2_emission', 'make_model')

Filling column: co2_emission
Grouping by: make_model
Number of NaN before filling: 486
Number of NaN filled: 485
Number of NaN after filling: 1
------------------
co2_emission
120.0    1141
97.0      793
99.0      762
114.0     653
119.0     622
         ... 
990.0       1
80.0        1
160.0       1
331.0       1
193.0       1
Name: count, Length: 120, dtype: int64


In [1192]:
df = fill("mode", df, 'co2_emission', 'body_type')

Filling column: co2_emission
Grouping by: body_type
Number of NaN before filling: 1
Number of NaN filled: 1
Number of NaN after filling: 0
------------------
co2_emission
120.0    1141
97.0      793
99.0      762
114.0     653
119.0     622
         ... 
165.0       1
80.0        1
160.0       1
331.0       1
193.0       1
Name: count, Length: 119, dtype: int64


### emission_class

In [1193]:
df = fill("mode", df, 'emission_class', 'co2_emission')

Filling column: emission_class
Grouping by: co2_emission
Number of NaN before filling: 3628
Number of NaN filled: 3620
Number of NaN after filling: 8
------------------
emission_class
Euro 6    15792
Euro 5       78
Euro 4       41
NaN           8
Name: count, dtype: int64


In [1194]:
df = fill("mode", df, 'emission_class', 'age')

Filling column: emission_class
Grouping by: age
Number of NaN before filling: 8
Number of NaN filled: 8
Number of NaN after filling: 0
------------------
emission_class
Euro 6    15800
Euro 5       78
Euro 4       41
Name: count, dtype: int64


### drive_chain

In [1195]:
df = fill("mode", df, 'drive_chain', ['make_model', 'body_type'])

Filling column: drive_chain
Grouping by: ['make_model', 'body_type']
Number of NaN before filling: 6858
Number of NaN filled: 6846
Number of NaN after filling: 12
------------------
drive_chain
front    15699
4WD        204
NaN         12
rear         4
Name: count, dtype: int64


In [1196]:
df = fill("mode", df, 'drive_chain', ['make_model'])

Filling column: drive_chain
Grouping by: ['make_model']
Number of NaN before filling: 12
Number of NaN filled: 12
Number of NaN after filling: 0
------------------
drive_chain
front    15711
4WD        204
rear         4
Name: count, dtype: int64


### entertainment_media

In [1104]:
df['entertainment_media']

0        'Bluetooth', 'Hands-free equipment', 'On-board...
1        'Bluetooth', 'Hands-free equipment', 'On-board...
2                               'MP3', 'On-board computer'
3        'Bluetooth', 'CD player', 'Hands-free equipmen...
4        'Bluetooth', 'CD player', 'Hands-free equipmen...
                               ...                        
15914    'Bluetooth', 'Digital radio', 'Hands-free equi...
15915    'Bluetooth', 'Digital radio', 'Hands-free equi...
15916    'Bluetooth', 'Hands-free equipment', 'On-board...
15917         'Bluetooth', 'Digital radio', 'Radio', 'USB'
15918                                                'USB'
Name: entertainment_media, Length: 15919, dtype: object

In [1105]:
df['entertainment_media'] = df['entertainment_media'].astype('str').str.replace('[','').str.replace("]",'')

#### This column was not changed as it will be transformed with getdummy function later

### safety_security

In [1106]:
df['safety_security']

0        ['ABS', 'Central door lock', 'Daytime running ...
1        ['ABS', 'Central door lock', 'Central door loc...
2        ['ABS', 'Central door lock', 'Daytime running ...
3        ['ABS', 'Alarm system', 'Central door lock wit...
4        ['ABS', 'Central door lock', 'Driver-side airb...
                               ...                        
15914    ['ABS', 'Central door lock', 'Central door loc...
15915    ['ABS', 'Adaptive Cruise Control', 'Blind spot...
15916    ['ABS', 'Adaptive Cruise Control', 'Blind spot...
15917    ['ABS', 'Blind spot monitor', 'Driver-side air...
15918    ['ABS', 'Blind spot monitor', 'Daytime running...
Name: safety_security, Length: 15919, dtype: object

#### This column was not changed as it will be transformed with getdummy function later

### comfort_convenience

In [1107]:
df['comfort_convenience']

0        'Air conditioning', 'Armrest', 'Automatic clim...
1        'Air conditioning', 'Automatic climate control...
2        'Air conditioning', 'Cruise control', 'Electri...
3        'Air suspension', 'Armrest', 'Auxiliary heatin...
4        'Air conditioning', 'Armrest', 'Automatic clim...
                               ...                        
15914    'Air conditioning', 'Automatic climate control...
15915    'Air conditioning', 'Automatic climate control...
15916    'Air conditioning', 'Armrest', 'Automatic clim...
15917    'Air conditioning', 'Automatic climate control...
15918    'Air conditioning', 'Automatic climate control...
Name: comfort_convenience, Length: 15919, dtype: object

### extras

In [1108]:
df['extras']

0        ['Alloy wheels', 'Catalytic Converter', 'Voice...
1        ['Alloy wheels', 'Sport seats', 'Sport suspens...
2                        ['Alloy wheels', 'Voice Control']
3         ['Alloy wheels', 'Sport seats', 'Voice Control']
4        ['Alloy wheels', 'Sport package', 'Sport suspe...
                               ...                        
15914                     ['Alloy wheels', 'Touch screen']
15915    ['Alloy wheels', 'Touch screen', 'Voice Control']
15916                                     ['Alloy wheels']
15917                     ['Alloy wheels', 'Touch screen']
15918                     ['Alloy wheels', 'Touch screen']
Name: extras, Length: 15919, dtype: object

---

## Quantitative Columns
- [x] price
- [x] km
- [] consumption_country
- [] consumption_city
- [] consumption_combined
- [] hp
- [] displacement
- [] warranty
- [] weight
- [] nr_of_doors
- [] nr_of_seats
- [] cylinders
- [] gears

### price

In [1109]:
df['price'].value_counts(dropna=False)

price
14990    154
15990    151
10990    139
15900    106
17990    102
        ... 
17559      1
17560      1
17570      1
17575      1
39875      1
Name: count, Length: 2956, dtype: int64

In [1110]:
df['price'].isnull().values.any()

False

### km

In [1111]:
df['km'].value_counts(dropna=False)

km
10.0       1045
NaN        1024
1.0         367
5.0         170
50.0        148
           ... 
67469.0       1
43197.0       1
10027.0       1
35882.0       1
57.0          1
Name: count, Length: 6690, dtype: int64

In [1112]:
fill('median', df, 'km', ['make_model', 'body_type', 'age'])

Filling column: km
Grouping by: ['make_model', 'body_type', 'age']
Number of NaN before filling: 1024
Number of NaN filled: 1020
Number of NaN after filling: 4
------------------
km
10.0        1277
1.0          367
100.0        197
750.0        181
5.0          174
            ... 
199000.0       1
157391.0       1
180573.0       1
87195.0        1
57.0           1
Name: count, Length: 6695, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,NaN,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,15.0,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


In [1113]:
fill('median', df, 'km', ['make_model', 'age'])

Filling column: km
Grouping by: ['make_model', 'age']
Number of NaN before filling: 1024
Number of NaN filled: 1024
Number of NaN after filling: 0
------------------
km
10.0        1356
100.0        376
1.0          367
50.0         298
73.5         238
            ... 
64976.0        1
144063.0       1
47625.0        1
154000.0       1
57.0           1
Name: count, Length: 6694, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,NaN,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,15.0,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


### consumption

 - #### comsumption column is filled with the mean of 'consumption_city','consumption_country' columns.

- #### Then is filled according to the column make_model.

### hp

In [1114]:
fill('median', df, 'hp', ['make_model', 'body_type', 'displacement'])

Filling column: hp
Grouping by: ['make_model', 'body_type', 'displacement']
Number of NaN before filling: 88
Number of NaN filled: 3
Number of NaN after filling: 85
------------------
hp
85.0     2542
66.0     2122
81.0     1402
100.0    1309
110.0    1113
         ... 
84.0        1
195.0       1
44.0        1
239.0       1
9.0         1
Name: count, Length: 81, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,NaN,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,NaN,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


### displacement

In [1115]:
df['displacement'].isnull().sum()

496

In [1116]:
fill("median", df, 'displacement', ['make_model', 'body_type', 'hp'])

Filling column: displacement
Grouping by: ['make_model', 'body_type', 'hp']
Number of NaN before filling: 496
Number of NaN filled: 403
Number of NaN after filling: 93
------------------
displacement
1598.0    4937
999.0     2465
1398.0    1345
1399.0     765
1229.0     708
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: count, Length: 80, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,NaN,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,NaN,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


### warranty

In [1117]:
df['warranty'].isnull().sum()

11066

In [1118]:
fill('median', df, 'warranty')

Filling column: warranty
Grouping by: None
Number of NaN before filling: 11066
Number of NaN filled: 11066
Number of NaN after filling: 0
------------------
warranty
12.0    13660
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
4.0         3
1.0         3
19.0        3
34.0        3
13.0        3
28.0        2
22.0        2
14.0        2
11.0        2
46.0        2
21.0        2
9.0         2
17.0        2
45.0        2
33.0        1
40.0        1
65.0        1
10.0        1
15.0        1
7.0         1
8.0         1
56.0        1
49.0        1
47.0        1
30.0        1
Name: count, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,12.0,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,12.0,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,12.0,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,12.0,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,12.0,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,NaN,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,12.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,12.0,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,12.0,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


### weight

In [1119]:
df['weight'].isnull().sum()

6974

In [1120]:
fill("median", df, 'weight', 'displacement')

Filling column: weight
Grouping by: displacement
Number of NaN before filling: 6974
Number of NaN filled: 6395
Number of NaN after filling: 579
------------------
weight
1403.0    1759
1163.0    1422
1195.0    1374
NaN        579
1308.0     523
          ... 
2295.0       1
1668.0       1
1428.0       1
2110.0       1
2037.0       1
Name: count, Length: 437, dtype: int64


,make_model,body_type,price,vat,km,hp,type,previous_owners,inspection_new,warranty,...,entertainment_media,extras,safety_security,gears,age,upholstery_material,particulate,combined_consumption,city_consumption,country_consumption
0,Audi A1,Sedans,15770,VAT deductible,56013.0,66.0,Used,2.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Catalytic Converter', 'Voice...","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000.0,141.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...","['Alloy wheels', 'Sport seats', 'Sport suspens...","['ABS', 'Central door lock', 'Central door loc...",7.0,2.0,Cloth,unparticulate,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450.0,85.0,Used,1.0,No,NaN,...,"'MP3', 'On-board computer'","['Alloy wheels', 'Voice Control']","['ABS', 'Central door lock', 'Daytime running ...",NaN,3.0,Cloth,unparticulate,3.8,4.4,3.4
3,Audi A1,Sedans,14500,NaN,73000.0,66.0,Used,1.0,No,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport seats', 'Voice Control']","['ABS', 'Alarm system', 'Central door lock wit...",6.0,3.0,NaN,unparticulate,3.8,4.3,3.5
4,Audi A1,Sedans,16790,NaN,16200.0,66.0,Used,1.0,Yes,NaN,...,"'Bluetooth', 'CD player', 'Hands-free equipmen...","['Alloy wheels', 'Sport package', 'Sport suspe...","['ABS', 'Central door lock', 'Driver-side airb...",NaN,3.0,Cloth,unparticulate,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,NaN,147.0,New,1.0,No,24.0,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen']","['ABS', 'Central door lock', 'Central door loc...",6.0,0.0,NaN,unparticulate,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900.0,165.0,Used,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Hands-free equi...","['Alloy wheels', 'Touch screen', 'Voice Control']","['ABS', 'Adaptive Cruise Control', 'Blind spot...",7.0,0.0,Cloth,unparticulate,NaN,7.0,NaN
15916,Renault Espace,Van,39875,VAT deductible,15.0,146.0,Pre-registered,1.0,Yes,NaN,...,"'Bluetooth', 'Hands-free equipment', 'On-board...",['Alloy wheels'],"['ABS', 'Adaptive Cruise Control', 'Blind spot...",6.0,0.0,Full leather,unparticulate,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10.0,147.0,Pre-registered,1.0,No,NaN,...,"'Bluetooth', 'Digital radio', 'Radio', 'USB'","['Alloy wheels', 'Touch screen']","['ABS', 'Blind spot monitor', 'Driver-side air...",6.0,0.0,Part leather,unparticulate,5.3,6.2,4.7


### nr_of_doors

In [1121]:
df['nr_of_doors'].value_counts()

nr_of_doors
5.0    11575
4.0     3079
3.0      832
2.0      219
1.0        1
7.0        1
Name: count, dtype: int64

### nr_of_seats

In [1122]:
df['nr_of_seats'].value_counts()

nr_of_seats
5.0    13336
4.0     1125
7.0      362
2.0      116
6.0        2
3.0        1
Name: count, dtype: int64

### cylinders

In [1123]:
df['cylinders'].value_counts(dropna=False)

cylinders
4.0    8105
NaN    5680
3.0    2104
5.0      22
6.0       3
8.0       2
2.0       2
1.0       1
Name: count, dtype: int64

### gears

In [1124]:
df['gears'].value_counts(dropna=False)

gears
6.0     5822
NaN     4712
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: count, dtype: int64

---

In [1125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   make_model            15919 non-null  object 
 1   body_type             15919 non-null  object 
 2   price                 15919 non-null  int64  
 3   vat                   11406 non-null  object 
 4   km                    14895 non-null  float64
 5   hp                    15831 non-null  float64
 6   type                  15917 non-null  object 
 7   previous_owners       15919 non-null  float64
 8   inspection_new        15919 non-null  object 
 9   warranty              4853 non-null   float64
 10  full_service          8215 non-null   object 
 11  non-smoking_vehicle   7177 non-null   object 
 12  null                  15919 non-null  object 
 13  offer_number          12744 non-null  object 
 14  first_registration    14322 non-null  object 
 15  body_color         

In [1126]:
# df.to_csv("", index=False)